In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import string

In [13]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### 1. Carga y limpieza inicial del corpus

Vamos a cargar nuestros archivos de 'tomatoes_reviews' en DataFrames de pandas, con el fin de poder manipularlos de una manera mucho más fácil y eficaz.

In [4]:
df_tomatoes_critics = pd.read_csv("tomatoes_data/rotten_tomatoes_critic_reviews.csv")
df_tomatoes_movies = pd.read_csv("tomatoes_data/rotten_tomatoes_movies.csv")

In [25]:
df_tomatoes_movies["rotten_tomatoes_link"] = df_tomatoes_movies["rotten_tomatoes_link"].astype("string")
df_tomatoes_movies["movie_title"] = df_tomatoes_movies["movie_title"].astype("string")

### 2. Preprocesamiento del corpus

Nuestro corpus está contenido en el archivo 'rotten_tomatoes_critic_reviews.csv' específicamente en la columna 'review_content'. Ahora lo que vamos a realizar es el preprocesamiento de este corpus, para ello vamos a realizar el siguiente proceso:

1. Extracción de contenido relevante de los documentos
2. Eliminar caracteres no deseados y normalizar el texto
3. Tokenización del texto
4. Eliminación de stop words y aplicación de lematización

Para este apartado vamos a crear una sola función de nombre preprocesar_contenido, la cual va a realizar todo el proceso de tokenización y normalización de cada contenido de cada documento que conforma el corpus

In [7]:
# Configuración de lematizador y stopwords
lemmatizer = WordNetLemmatizer()

# Carga de stopwords en inglés y signos de puntuación para limpiar
stopwords = set(stopwords.words('english'))
signos_puntuacion = string.punctuation

In [15]:
# Función de preprocesamiento (sin etiquetado POS)
def preprocesar_contenido(contenido):
    # Convertir texto a minúsculas
    contenido = contenido.lower()

    # Eliminar caracteres redundantes y limpiar texto
    contenido = re.sub(r'\d+', '', contenido)  # Eliminar números
    contenido = re.sub(r'\s+', ' ', contenido)  # Eliminar saltos de línea y espacios redundantes

    # Remover signos de puntuación
    contenido = contenido.translate(str.maketrans("", "", signos_puntuacion))

    # Tokenización
    contenido_tokenizado = word_tokenize(contenido)

    # Eliminar stopwords
    tokens_sin_stopwords = [word for word in contenido_tokenizado if word not in stopwords]

    # Lematización asumiendo sustantivos por defecto
    tokens_lematizados = [lemmatizer.lemmatize(token, wordnet.NOUN) for token in tokens_sin_stopwords]

    # Reconstruir texto lematizado a un solo string
    contenido_lematizado_str = " ".join(tokens_lematizados)

    return contenido_lematizado_str

In [16]:
# Conversión entre tipo de datos para evitar problemas
df_tomatoes_critics
df_tomatoes_critics["review_content"] = df_tomatoes_critics["review_content"].astype("string")
df_tomatoes_critics = df_tomatoes_critics.dropna(subset=["review_content"])

In [17]:
# Preprocesar todo el corpus
for index, row in df_tomatoes_critics.iterrows():
    print(f"Iterando sobre elemento {index+1} del .csv de los corpus")
    df_tomatoes_critics.at[index, "contenido_preproc_str"] = preprocesar_contenido(row["review_content"])

Streaming output truncated to the last 5000 lines.
Iterando sobre elemento 1124829 del .csv de los corpus
Iterando sobre elemento 1124830 del .csv de los corpus
Iterando sobre elemento 1124831 del .csv de los corpus
Iterando sobre elemento 1124832 del .csv de los corpus
Iterando sobre elemento 1124834 del .csv de los corpus
Iterando sobre elemento 1124835 del .csv de los corpus
Iterando sobre elemento 1124836 del .csv de los corpus
Iterando sobre elemento 1124837 del .csv de los corpus
Iterando sobre elemento 1124838 del .csv de los corpus
Iterando sobre elemento 1124839 del .csv de los corpus
Iterando sobre elemento 1124840 del .csv de los corpus
Iterando sobre elemento 1124841 del .csv de los corpus
Iterando sobre elemento 1124842 del .csv de los corpus
Iterando sobre elemento 1124843 del .csv de los corpus
Iterando sobre elemento 1124844 del .csv de los corpus
Iterando sobre elemento 1124845 del .csv de los corpus
Iterando sobre elemento 1124846 del .csv de los corpus
Iterando sobre

In [19]:
# Guardar corpus en un nuevo archivo .csv
df_tomatoes_critics.to_csv("tomatoes_data/rotten_tomatoes_critic_reviews_preproc.csv", index=False)

### 3. Representación de datos en espacio vectorial:

Vamos a utilizar TF-IDF para vectorizar nuestro corpus y representarlo como un espacio vectorial.

La función implementada para este fin devolverá el corpus vectorizado y el vectorizador usado.

In [20]:
# Lectura del nuevo archivo .csv de tomatoes preprocesado

df_tomatoes_critics_preproc = pd.read_csv("tomatoes_data/rotten_tomatoes_critic_reviews_preproc.csv")
df_tomatoes_critics_preproc["review_content"] = df_tomatoes_critics_preproc["review_content"].astype("string")
df_tomatoes_critics_preproc["contenido_preproc_str"] = df_tomatoes_critics_preproc["contenido_preproc_str"].astype("string")
df_tomatoes_critics_preproc = df_tomatoes_critics_preproc.dropna(subset=["contenido_preproc_str"])

In [21]:
# ---- TF-IDF ----
def vectorize_tfidf(corpus):
    """
    Vectoriza un corpus utilizando TF-IDF.
    """
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)
    return X, vectorizer

In [24]:
# Vectorizar el corpus con TF-IDF
print("Vectorizando con TF-IDF")
X_tfidf, tfidf_vectorizer = vectorize_tfidf(df_tomatoes_critics_preproc["contenido_preproc_str"])

Vectorizando con TF-IDF


### 4. Búsqueda de una consulta utilizando distancia coseno

Vamos a implementar una función para realizar la búsqueda, dicha función recibirá la consulta, el corpus vectorizado y el vectorizador TF-IDF utilizado.

1. Preprocesará la consulta
2. Vectorizará la consulta
3. Calculará similitud coseno respecto al corpus
4. Devolverá los resultados relevantes

In [26]:
def buscar_documentos(consulta, corpus_vectorizado, vectorizador, metodo="tfidf"):
    """
    Busca documentos en el corpus y devuelve los más relevantes según el método elegido (BoW, TF-IDF, Word2Vec).
    """

    # Preprocesar la consulta
    consulta = preprocesar_contenido(consulta)

    # Vectorizar la consulta
    consulta_vec = vectorizador.transform([consulta])

    # Calcular similitudes
    similitudes = cosine_similarity(consulta_vec, corpus_vectorizado)
    resultados = similitudes[0]

    # Rankear documentos por relevancia y tomar los top_n
    indices_ordenados = resultados.argsort()[::-1]

    return indices_ordenados, resultados

In [27]:
# Definir la consulta y el máximo de resultados a mostrar
consulta = "Harry Potter"
max_resultados = 20

In [33]:
# ---- Obtener índices ordenados y hacer merge con df_tomatoes_movies ----
indices_ordenados, resultados = buscar_documentos(consulta, X_tfidf, tfidf_vectorizer)

# Realizar un merge entre los DataFrames basado en rotten_tomatoes_link
df_combined = pd.merge(
    df_tomatoes_critics_preproc,
    df_tomatoes_movies[["rotten_tomatoes_link", "movie_title"]],
    on="rotten_tomatoes_link",
    how="left"
)

# Ahora puedes usar df_combined para acceder tanto a las reseñas como a los títulos
resultados_detallados = [
    {
        "Película": df_combined.loc[indice, "movie_title"],  # Título de la película
        "Relevancia": resultados[indice],  # Relevancia calculada
        "Contenido reseña": df_combined.loc[indice, "review_content"],  # Contenido de la reseña
    }
    for indice in indices_ordenados
]

In [34]:
# Finalmente, mostrar los resultados
print("")
print(f"Mostrando resultados relevantes para la búsqueda:   {consulta}")
print("")
for resultado in resultados_detallados[:max_resultados]:
    print(f"\nPelícula: {resultado['Película']}, \nRelevancia: {resultado['Relevancia']:.3f}")
    print(f"Reseña: {resultado['Contenido reseña']}\n")
    print("=====================================================================================================================")


Mostrando resultados relevantes para la búsqueda:   Harry Potter


Película: Harry Potter and the Half-Blood Prince, 
Relevancia: 0.830
Reseña: Harry Potter grows up. Harry Potter needs to shave. Harry Potter shows more interest in the young ladies at Hogwarts.


Película: Harry Potter and the Prisoner of Azkaban, 
Relevancia: 0.820
Reseña: Not only is this the best Harry Potter movie, it is the first Harry Potter movie that actually qualifies as cinema.


Película: Doctor Strange, 
Relevancia: 0.804
Reseña: The movie is very much Harry Potter for adults.


Película: Harry Potter and the Half-Blood Prince, 
Relevancia: 0.802
Reseña: Harry Potter and the Half-Blood Prince might be the best Harry Potter film yet.


Película: Harry Potter and the Order of the Phoenix, 
Relevancia: 0.801
Reseña: The second best Harry Potter movie.


Película: Harry Potter and the Prisoner of Azkaban, 
Relevancia: 0.788
Reseña: This is the best Harry Potter film to date.


Película: Harry Potter and the Go

rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,streaming_release_date,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_crit